# Using UDFs in PySpark

In [0]:
display(dbutils.fs.ls("abfss://etl1@dbstoragebbpbs73u57xmm.dfs.core.windows.net/Exercise2/"))

# Let's load our streaming events data
file_path = "abfss://etl1@dbstoragebbpbs73u57xmm.dfs.core.windows.net/Exercise2/complex_events.csv"
df = spark.read.option("header", "true").option("inferSchema", "true").csv(file_path)

### USER DEFINED FUNCTIONS (UDFs)

In [0]:
# UDFs allow us to apply custom Python functions to our data

from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, IntegerType, FloatType

# Let's create a simple UDF to calculate streaming quality score
@udf(FloatType())
def calculate_quality_score(duration, buffering_count):
    if duration is None or buffering_count is None:
        return None
    
    # Calculate a simple score where longer duration is good but buffering is bad
    # Higher score is better
    return (duration / 60.0) - (buffering_count * 10.0)

# Apply our UDF to the DataFrame
df_with_quality = df.withColumn("streaming_score", 
                               calculate_quality_score(df["duration_seconds"], df["buffering_count"]))

print("Data with streaming scores (Regular UDF):")
df_with_quality.select("duration_seconds", "buffering_count", "streaming_score").show(5)


In [0]:
# UDFs can significantly slow down processing since they operate row by row
# Let's check the execution plan to see the impact
print("Execution plan with Regular UDF:")
print("--------------------------------")
df_with_quality.explain(mode="formatted")

### VECTORIZED UDFs

In [0]:
# VECTORIZED UDFs
# Vectorized UDFs process data in batches using pandas, which dramatically improves performance

import pandas as pd
from pyspark.sql.functions import pandas_udf

# Define a vectorized UDF for calculating the SAME streaming score
# But using pandas for vectorized processing
@pandas_udf(FloatType())
def calculate_quality_score_vectorized(duration: pd.Series, buffering: pd.Series) -> pd.Series:
    # Same logic as the regular UDF, but operates on entire Series at once
    # This uses pandas' optimized C implementation for fast vector operations
    return (duration / 60.0) - (buffering * 10.0)

# Apply the vectorized UDF
df_with_score_vectorized = df.withColumn("streaming_score", 
                             calculate_quality_score_vectorized(df["duration_seconds"], df["buffering_count"]))

print("Data with streaming scores (Vectorized UDF):")
df_with_score_vectorized.select("duration_seconds", "buffering_count", "streaming_score").show(5)

In [0]:
# Let's see the execution plan difference with vectorized UDF
print("\nExecution plan with Vectorized (Pandas) UDF:")
print("-------------------------------------------")
df_with_score_vectorized.explain(mode="formatted")

In [0]:
# Comparing performance between regular UDF and vectorized UDF
import time

#spark.config.set("spark.databricks.io.cache.enabled","true")

# Force Spark to clear any previous caches or plans
spark.catalog.clearCache()

# Time the regular UDF
start_time = time.time()
result1 = df_with_quality.select("streaming_score")  # Use the DataFrame we already created with regular UDF
result1.count()  # Force execution
regular_time = time.time() - start_time
print(f"Regular UDF execution time: {regular_time:.2f} seconds")

# Time the vectorized UDF
start_time = time.time()
result2 = df_with_score_vectorized.select("streaming_score")  # Use the DataFrame we already created with vectorized UDF
result2.count()  # Force execution
vectorized_time = time.time() - start_time
print(f"Vectorized UDF execution time: {vectorized_time:.2f} seconds")

print(f"Speedup factor: {regular_time/vectorized_time:.2f}x faster with vectorized UDF")


In [0]:
# Key differences in the explain output:
# 1. "ArrowEvalPython" instead of "BatchEvalPython" - uses Apache Arrow for efficient data transfer
# 2. Fewer conversion operations than regular UDFs
# 3. Better integration with Spark's optimization engine
# Note: Both UDF types still break Photon optimization, but vectorized UDFs are much more efficient